In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from scipy.sparse import csr_matrix
import scipy.sparse as sp

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv") 
X = data.drop("target", axis=1) 
y = data['target']

In [3]:
X_test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv") 

Preprocessing for specfific fields:

In [4]:
l1=[]
for i in data.DateAS:
  l1.append(i.split(' ')[0])
data['DateAS']=l1
X['DateAS']=l1

In [5]:
X['SignatureVersion']=X['SignatureVersion'].apply(lambda x: x.split('.')[1])
X['OSVersion']=X['OSVersion'].apply(lambda x: x.split('.')[0])

In [6]:
#processing the dateAS column
#l1=[]

#for i in data.DateAS:
#  l1.append(i.split(' ')[0])
#data['DateAS']=l1
#X['DateAS']=l1

#dropping the unnecessary columns 
cols_to_drop=['MachineID', 'IsBetaUser',
              'CityID', 'AutoSampleSubmissionEnabled',
               'OEMModelID','IsFlightsDisabled', 'FirmwareVersionID', 'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OsPlatformSubRelease',
               'OSBuildNumber', 'OSBranch', 'OSArchitecture', 'OSSkuFriendlyName'
              ]

#creating a df with the reqd columns
X_new=X.drop(cols_to_drop,axis=1)
new_columns=X_new.columns

#imputing the nulll values with most frequent values
for i in X_new:
  if ("UNKNOWN" in X_new[i].unique()) or ('Unknown' in X_new[i].unique()) or ("Unspecified" in X_new[i].unique()):
    X_new[i].replace('Unknown',np.nan,inplace=True)
    X_new[i].replace('UNKNOWN',np.nan,inplace=True)
    X_new[i].replace('Unspecified',np.nan,inplace=True)

#processing the EnableLUA field:
X_new['EnableLUA'].value_counts()
X_new['EnableLUA'].replace(48,1,inplace=True)


#temp=X_new.isna().sum()
#cols_to_impute=temp[temp>0].index
si=SimpleImputer(strategy='most_frequent')
X_new_si=si.fit_transform(X_new)

X_new=pd.DataFrame(X_new_si, columns=X_new.columns)






Combining chassistype and *MDC2Formfactor*

Combining HasOpticalDiskDrive and PrimmaryDiskType


In [7]:
X_new['HasOptical_PrimaryDiskType']=X_new['HasOpticalDiskDrive'].astype(str)+'_'+X_new['PrimaryDiskType']

In [8]:
X_new.drop(['HasOpticalDiskDrive','PrimaryDiskType'],axis=1,inplace=True)

In [9]:
X_new.shape

(100000, 60)

In [10]:
type(X_new)

pandas.core.frame.DataFrame

In [11]:
X_new['Number_of_Pixels']=X_new['PrimaryDisplayResolutionHorizontal']*X_new['PrimaryDisplayResolutionVertical']
X_new.drop(['PrimaryDisplayResolutionHorizontal','PrimaryDisplayResolutionVertical'],axis=1,inplace=True)

In [12]:
#numeric fields
X_numeric=X_new[['SystemVolumeCapacityMB', 'PrimaryDiskCapacityMB', 'ProcessorCoreCount',
                 'TotalPhysicalRAMMB', 'PrimaryDisplayDiagonalInches', 'Number_of_Pixels',
                 'InternalBatteryNumberOfCharges']]
# The field NumAntivirusProductsInstalled is numeric but does not need scaling
#scaling the other numeric fields:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_numeric_scaled=ss.fit_transform(X_numeric)
X_numeric_scaled=pd.DataFrame(X_numeric_scaled,columns=X_numeric.columns)


In [13]:
X_categorical=X_new.drop(X_numeric.columns,axis=1)

In [14]:
X_categorical.shape

(100000, 52)

**PREPROCESSING THE TEST FIELDS UPFRONT**

In [15]:
#doing the same preprocessing for the X_test
#processing the dateAS column

l1=[]
for i in X_test.DateAS:
  l1.append(i.split(' ')[0])
X_test['DateAS']=l1


X_test['SignatureVersion']=X_test['SignatureVersion'].apply(lambda x: x.split('.')[1])
X_test['OSVersion']=X_test['OSVersion'].apply(lambda x: x.split('.')[0])
#X_test['AppVersion']=X_test['AppVersion'].apply(lambda x: x.split('.')[1])
#X_test['EngineVersion']=X_test['EngineVersion'].apply(lambda x: x.split('.')[2])




#dropping the unnecessary columns 
cols_to_drop=['MachineID', 'IsBetaUser',
              'CityID', 'AutoSampleSubmissionEnabled',
               'OEMModelID','IsFlightsDisabled', 'FirmwareVersionID', 'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OsPlatformSubRelease',
               'OSBuildNumber', 'OSBranch', 'OSArchitecture', 'OSSkuFriendlyName'
              ]

#creating a df with the reqd columns
X_new_test=X_test.drop(cols_to_drop,axis=1)
new_columns_test=X_new_test.columns

#imputing the nulll values with most frequent values
for i in X_new_test:
  if ("UNKNOWN" in X_new_test[i].unique()) or ('Unknown' in X_new_test[i].unique()) or ("Unspecified" in X_new_test[i].unique()):
    X_new_test[i].replace('Unknown',np.nan,inplace=True)
    X_new_test[i].replace('UNKNOWN',np.nan,inplace=True)
    X_new_test[i].replace('Unspecified',np.nan,inplace=True)







#processing the EnableLUA field:
X_new_test['EnableLUA'].value_counts()
X_new_test['EnableLUA'].replace(48,1,inplace=True)

#temp=X_new_test.isna().sum()
#cols_to_impute=temp[temp>0].index
X_new_si_test=si.transform(X_new_test)

X_new_test=pd.DataFrame(X_new_si_test, columns=X_new_test.columns)

X_new_test['Number_of_Pixels']=X_new_test['PrimaryDisplayResolutionHorizontal']*X_new_test['PrimaryDisplayResolutionVertical']
X_new_test.drop(['PrimaryDisplayResolutionHorizontal','PrimaryDisplayResolutionVertical'],axis=1,inplace=True)




#Combining HasOpticalDiskDrive and PrimaryDiskType for the test set

X_new_test['HasOptical_PrimaryDiskType']=X_new_test['HasOpticalDiskDrive'].astype(str)+'_'+X_new_test['PrimaryDiskType']
X_new_test.drop(['HasOpticalDiskDrive','PrimaryDiskType'],axis=1,inplace=True)



#numeric test fields:


X_numeric_test=X_new_test[['SystemVolumeCapacityMB', 'PrimaryDiskCapacityMB', 'ProcessorCoreCount',
                 'TotalPhysicalRAMMB', 'PrimaryDisplayDiagonalInches','Number_of_Pixels',
                 'InternalBatteryNumberOfCharges']]
# The field NumAntivirusProductsInstalled is numeric but does not need scaling
#scaling the other numeric fields:

X_numeric_scaled_test=ss.transform(X_numeric_test)
X_numeric_scaled_test=pd.DataFrame(X_numeric_scaled_test,columns=X_numeric_test.columns)


#make sure all the fields in x_numeric scaled are numeric test
for i in X_numeric_scaled_test.columns:
  X_numeric_scaled_test[i]=pd.to_numeric(X_numeric_scaled_test[i])





#categorical fields test:
X_categorical_test=X_new_test.drop(X_numeric_test.columns,axis=1)


In [16]:
#X_test_categorical_concat.shape

**SELECTING CATEGORICAL FIELDS ACCORDING TO THE NUMBER OF UNIQUE VALUES PRESENT**

TRAIN SET

Target Encoding selected columns:

In [17]:
from category_encoders import TargetEncoder
targetencoder=TargetEncoder()
X_targetenc=targetencoder.fit_transform(X_categorical[['MDC2FormFactor','ChassisType', 'RealTimeProtectionState', 'PowerPlatformRole', 'AutoUpdateOptionsName', 'NumAntivirusProductsInstalled', 'Processor', 'OSInstallType', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'HasOptical_PrimaryDiskType', 'SKUEditionName', 'SMode', 'IsSystemProtected', 'DeviceFamily', 'FirewallEnabled', 'IsPortableOS', 'IsVirtualDevice', 'IsTouchEnabled', 'IsPenCapable', 'IsGamer', 'IsAlwaysOnAlwaysConnectedCapable'] ], y)

In [18]:
X_targetenc=pd.DataFrame(X_targetenc,columns=['MDC2FormFactor','ChassisType', 'RealTimeProtectionState', 'PowerPlatformRole', 'AutoUpdateOptionsName', 'NumAntivirusProductsInstalled', 'Processor', 'OSInstallType', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'HasOptical_PrimaryDiskType', 'SKUEditionName', 'SMode', 'IsSystemProtected', 'DeviceFamily', 'FirewallEnabled', 'IsPortableOS', 'IsVirtualDevice', 'IsTouchEnabled', 'IsPenCapable', 'IsGamer', 'IsAlwaysOnAlwaysConnectedCapable'])

ONEHOT ENCODINFG

In [19]:
#onehot encoding:
onehot=OneHotEncoder(handle_unknown='ignore')
X_categorical=X_categorical.drop(X_targetenc.columns,axis=1)
X_categorical_encoded=onehot.fit_transform(X_categorical)

Appending the targetenc df with the numeric df

In [20]:
X_numeric_scaled=pd.concat([X_numeric_scaled,X_targetenc],axis=1)

In [21]:
#converting x_numeric to sparse:
from scipy.sparse import csr_matrix
X_numeric_scaled=csr_matrix(X_numeric_scaled)

TEST SET

In [22]:
X_targetenc_test=targetencoder.transform(X_categorical_test[['MDC2FormFactor','ChassisType', 'RealTimeProtectionState', 'PowerPlatformRole', 'AutoUpdateOptionsName', 'NumAntivirusProductsInstalled', 'Processor', 'OSInstallType', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'HasOptical_PrimaryDiskType', 'SKUEditionName', 'SMode', 'IsSystemProtected', 'DeviceFamily', 'FirewallEnabled', 'IsPortableOS', 'IsVirtualDevice', 'IsTouchEnabled', 'IsPenCapable', 'IsGamer', 'IsAlwaysOnAlwaysConnectedCapable'] ])

In [23]:
X_targetenc_test=pd.DataFrame(X_targetenc_test,columns=['MDC2FormFactor','ChassisType', 'RealTimeProtectionState', 'PowerPlatformRole', 'AutoUpdateOptionsName', 'NumAntivirusProductsInstalled', 'Processor', 'OSInstallType', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'HasOptical_PrimaryDiskType', 'SKUEditionName', 'SMode','IsSystemProtected', 'DeviceFamily', 'FirewallEnabled', 'IsPortableOS', 'IsVirtualDevice', 'IsTouchEnabled', 'IsPenCapable', 'IsGamer', 'IsAlwaysOnAlwaysConnectedCapable'])

Onehot encoding the test set

In [24]:
#onehot encoding:
X_categorical_test=X_categorical_test.drop(X_targetenc_test.columns,axis=1)
X_categorical_encoded_test=onehot.transform(X_categorical_test)

In [25]:
X_numeric_scaled_test=pd.concat([X_numeric_scaled_test,X_targetenc_test],axis=1)

In [26]:
#converting x_numeric to sparse:
from scipy.sparse import csr_matrix
X_numeric_scaled_test=csr_matrix(X_numeric_scaled_test)

Making sure all the values are numeric

In [27]:
X_concat=sp.hstack((X_numeric_scaled,X_categorical_encoded))

In [28]:
X_concat_test=sp.hstack((X_numeric_scaled_test,X_categorical_encoded_test))

In [29]:
X_concat.shape, X_concat_test.shape

((100000, 6901), (10000, 6901))

COMBINING THE CATEGORICAL AND THE NUMERIC COLUMNS:

In [30]:
X_concat_test.shape

(10000, 6901)

In [31]:
from scipy.sparse import csr_matrix

In [32]:
#apply Light GBM:

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold


train_data = lgb.Dataset(X_concat, label=y)

'''
# Defining the parameters
params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'max_depth': 7,
    'min_child_samples': 50,
    'n_estimators': 200,
    'num_leaves': 31,
    'subsample': 1,
    #'lambda_l1': 0.1,
    #'lambda_l2': 0.1
}

# Create the LightGBM classifier with the best parameters
model = lgb.LGBMClassifier(**params, objective='binary')

# Train the model on the sparse feature matrix X_concat and label vector y
model.fit(X_concat, y)

'''



# Defining the best parameters
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.05,
    'min_child_samples': 20,
    'n_estimators': 200,
    'num_leaves': 63,
    'max_depth' : -1,
    'subsample': 1
}

# Create the LightGBM classifier with the best parameters
model = lgb.LGBMClassifier(**best_params, objective='binary')  

# Train the model on the sparse feature matrix X_concat and label vector y
model.fit(X_concat, y) 



[LightGBM] [Info] Number of positive: 50525, number of negative: 49475
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.334408 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5138
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 2028
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505250 -> initscore=0.021001
[LightGBM] [Info] Start training from score 0.021001


LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, n_estimators=200,
               num_leaves=63, objective='binary', subsample=1)

In [33]:
y_pred=model.predict(X_concat_test)

In [34]:
submission = pd.DataFrame({"id": range(0,X_test.shape[0]), "target": y_pred})
submission.to_csv('submission.csv',index=False)

In [35]:
X_concat_test.shape

(10000, 6901)

In [36]:
X_test.shape

(10000, 75)